In [1]:
from re import match
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import pickle
from sklearn.model_selection import train_test_split

2025-03-26 19:41:12.955184: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-26 19:41:12.968102: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743036072.981101 1998765 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743036072.984812 1998765 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-26 19:41:12.999887: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
csv_plays = '/home/research/Big Data Bowl/raw_data_folders/nfl-big-data-bowl-2025/plays.csv'
play_data = pd.read_csv(csv_plays)

In [3]:
for clm in ['dropbackType','rushLocationType','pff_runConceptPrimary','pff_passCoverage','pff_manZone']:
    play_data[clm] = np.where(play_data['qbKneel'] == 1, 'QB_KNEEL', play_data[clm])
    play_data[clm] = np.where(play_data['qbSpike'] == True, 'QB_SPIKE', play_data[clm])
play_data['offenseFormation'] = np.where((play_data['offenseFormation'].isna()) & (play_data['qbKneel'] == 1), 'QB_KNEEL', play_data['offenseFormation'])
play_data['offenseFormation'] = np.where((play_data['offenseFormation'].isna()) & (play_data['qbSpike'] == True), 'QB_SPIKE', play_data['offenseFormation'])
play_data['dropbackType'] = np.where(play_data['qbSneak'] == True, 'QB_SNEAK', play_data['dropbackType'])
play_data['dropbackType'] = np.where((play_data['qbKneel'] == 0) & (play_data['qbSneak'] == False) & ((play_data['dropbackType'].isna()) | (play_data['dropbackType'] == "UNKNOWN")), 'NON_QB_RUN', play_data['dropbackType'])
play_data['playType'] = np.where(play_data['rushLocationType'].isna(), play_data['dropbackType'].astype(str), play_data['dropbackType'].astype(str) + '/' + play_data['rushLocationType'].astype(str))
play_data['receiverAlignment'] = np.where(play_data['receiverAlignment'].isna(), '0x0', play_data['receiverAlignment'].astype(str))
play_data['playAction'] = np.where(play_data['playAction'] == True, 1, 0)
play_data['pff_passCoverage'] = np.where((play_data['qbSneak'] == True) & (play_data['pff_passCoverage'].isna()), 'QB_SNEAK', play_data['pff_passCoverage'])
play_data = play_data[play_data['pff_passCoverage'] != 'Miscellaneous']
play_data = play_data[play_data['pff_passCoverage'].notna()]
play_data['pff_runConceptPrimary'] = np.where(((play_data['pff_runConceptPrimary'].isna()) | (play_data['pff_runConceptPrimary'] == 'UNDEFINED')) & (play_data['dropbackType'] != 'UNKNOWN') & (play_data['dropbackType'].notna()), 'DESIGNED_PASS', play_data['pff_runConceptPrimary'])
play_data['runConcept'] = np.where(play_data['pff_runConceptSecondary'].isna(), play_data['pff_runConceptPrimary'].astype(str), play_data['pff_runConceptPrimary'].astype(str) + ';' + play_data['pff_runConceptSecondary'].astype(str))
play_data['offensiveSuccess'] = np.where(((play_data['down'] == 1) | (play_data['down'] == 2)) & (play_data['prePenaltyYardsGained'] >= play_data['yardsToGo']/2), 1, 0)
play_data['offensiveSuccess'] = np.where(((play_data['down'] == 3) | (play_data['down'] == 4)) & (play_data['prePenaltyYardsGained'] >= play_data['yardsToGo']), 1, 0)
play_data['game_play'] = play_data['gameId'].astype(str) + '_' + play_data['playId'].astype(str)
play_data2 = play_data[['game_play','playAction','pff_runPassOption','offenseFormation','receiverAlignment','playType','pff_passCoverage','runConcept','offensiveSuccess']]
play_data3 = play_data2.rename(columns={'playAction': 'isPA', 'pff_runPassOption': 'isRPO', 'offenseFormation': 'off_form', 'receiverAlignment': 'rec_form','pff_passCoverage': 'passCov', 'offensiveSuccess': 'off_scs'})


In [4]:
labels_of = {value: index for index, value in enumerate(list(np.unique(play_data3['off_form'])))}
labels_rf = {value: index for index, value in enumerate(list(np.unique(play_data3['rec_form'])))}
labels_pt = {value: index for index, value in enumerate(list(np.unique(play_data3['playType'])))}
labels_pc = {value: index for index, value in enumerate(list(np.unique(play_data3['passCov'])))}
labels_rc = {value: index for index, value in enumerate(list(np.unique(play_data3['runConcept'])))}
play_data3['of_norm'] = play_data3['off_form'].map(labels_of)
play_data3['rf_norm'] = play_data3['rec_form'].map(labels_rf)
play_data3['pt_norm'] = play_data3['playType'].map(labels_pt)
play_data3['pc_norm'] = play_data3['passCov'].map(labels_pc)
play_data3['rc_norm'] = play_data3['runConcept'].map(labels_rc)


In [5]:
players = pd.read_csv('/home/research/Big Data Bowl/raw_data_folders/nfl-big-data-bowl-2025/players.csv')

In [6]:
game_plays = pd.DataFrame()
for wk in np.arange(9)+1:
    gp = pd.read_csv('/home/research/Big Data Bowl/raw_data_folders/nfl-big-data-bowl-2025/tracking_week_' + str(wk) + '.csv')
    game_plays = pd.concat([game_plays, gp], ignore_index=True)

In [7]:
game_plays['game_play'] = game_plays['gameId'].astype(str) + '_' + game_plays['playId'].astype(str)
gameplays0 = pd.merge(game_plays, players[['nflId','position']], how='left', on='nflId')
gameplays1 = pd.merge(gameplays0, play_data[['game_play','possessionTeam','yardlineSide','yardlineNumber','yardsToGo']], how='left', on='game_play')


In [8]:
#assess the number of frames before and after snap
gpx = gameplays1[gameplays1['displayName'] == 'football'].groupby(['game_play','frameType']).size()
# gpxw = pd.pivot_table(pd.DataFrame(gpx), index='game_play', columns='frameType')[0]


In [9]:
gameplays2 = gameplays1[['game_play','frameId','position','playDirection','x','y','club','possessionTeam','displayName']]
gameplays2 = gameplays2.rename(columns={'x': 'x_pos', 'y': 'y_pos'})
gameplays2.loc[(gameplays2['playDirection'] == 'left'), 'x_pos'] = 120 - gameplays2['x_pos']
gameplays2.loc[(gameplays2['playDirection'] == 'left'), 'y_pos'] = (160/3) - gameplays2['y_pos']
# gameplays2['side'] = np.where(gameplays2['club'] == gameplays2['possessionTeam'], 'QB_SNEAK', play_data['pff_passCoverage'])
gameplays2['side'] = 'other'
gameplays2.loc[(gameplays2['club'] == gameplays2['possessionTeam']) & (gameplays2['displayName'] != 'football'), 'side'] = 'off'
gameplays2.loc[(gameplays2['club'] != gameplays2['possessionTeam']) & (gameplays2['displayName'] != 'football'), 'side'] = 'def'
gameplays2.loc[gameplays2['displayName'] == 'football','side'] = 'fb'
gameplays2.loc[gameplays2['side'] == 'fb', 'position'] = 'FTBL'

In [10]:
#remove plays with offensive and defensive position mismatches (e.g., DT on offense)
mismatch_off = gameplays2.loc[(gameplays2['position'].isin(['G','T','TE','C','WR','QB','RB','FB'])) & (gameplays2['side'] == 'def'),'game_play'].unique()
mismatch_def = gameplays2.loc[(gameplays2['position'].isin(['OLB','DT','ILB','FS'])) & (gameplays2['side'] == 'off'),'game_play'].unique()
mismatch_tot = mismatch_def.tolist() + mismatch_off.tolist()
gameplays3 = gameplays2.drop(gameplays2.loc[gameplays2["game_play"].isin(mismatch_tot)].index)
pos_groups = gameplays3.groupby('side')['position'].unique()

In [11]:
pos_codes = {'QB':2, 'G':3, 'WR':4, 'C':5, 'RB':6, 'TE':7, 'T':8, 'FB':9, 
                'SS':-2, 'CB':-3, 'OLB':-4, 'DE':-5, 'ILB':-6, 'FS':-7, 'NT':-8, 
                'DT':-9, 'MLB':-10, 'LB':-11, 'DB':-12, 'FTBL':50}
gameplays3['pos_id'] = gameplays3['position'].map(pos_codes)
gameplays3['col_idx'] = round(100*gameplays3['x_pos']/120).astype(int)
gameplays3['row_idx'] = round(100*gameplays3['y_pos']/(160/3)).astype(int)

In [12]:
gameplays3 = pd.merge(gameplays3, gameplays3[(gameplays3['position'] == 'FTBL') & (gameplays3['frameId'] == 1)][['game_play','col_idx']], how='left', on='game_play')
gameplays3['col_norm'] = gameplays3['col_idx_x'] + (50 - gameplays3['col_idx_y'])


In [13]:
# Select only relevant features
gameplays3 = gameplays3[['game_play', "frameId", 'pos_id', 'row_idx', 'col_norm']]
under_thresh = gameplays3.groupby('game_play')['frameId'].nunique()
thresh_ids = under_thresh[under_thresh <= 350].index.to_list()     
gameplays4 = gameplays3[gameplays3['game_play'].isin(thresh_ids)]
play_data4 = play_data3[play_data3['game_play'].isin(thresh_ids)]

In [14]:
print(len(np.unique(gameplays4['game_play'])))
print(len(np.unique(play_data4['game_play'])))

16059
16059


In [15]:
import os
import numpy as np
from sklearn.model_selection import train_test_split

# Split train and test data
trainplays, testplays = train_test_split(
    np.unique(play_data4['game_play']), 
    train_size=0.8, 
    random_state=1, 
    shuffle=True
)

# Create a single folder to store all files
save_dir = "splits"
os.makedirs(save_dir, exist_ok=True)

# Save tracking data
gameplays4[gameplays4['game_play'].isin(trainplays)].to_pickle(os.path.join(save_dir, 'tracking_processed_train.pkl'))
gameplays4[gameplays4['game_play'].isin(testplays)].to_pickle(os.path.join(save_dir, 'tracking_processed_test.pkl'))
print('Tracking data saved!')

# Save label data
play_data4[play_data4['game_play'].isin(trainplays)].to_pickle(os.path.join(save_dir, 'label_processed_train.pkl'))
play_data4[play_data4['game_play'].isin(testplays)].to_pickle(os.path.join(save_dir, 'label_processed_test.pkl'))
print('Label data saved!')


Tracking data saved!
Label data saved!


In [16]:
print("The files were made successfully and they were saved in train test splits.")

The files were made successfully and they were saved in train test splits.
